In [ ]:
import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D, Flatten, Dropout, MaxPooling2D
from keras.applications.mobilenet import preprocess_input
from keras.applications.mobilenet_v2 import MobileNetV2
import numpy as np
from IPython.display import Image
from keras.optimizers import Adam
import tensorflow as tf

In [ ]:
!unzip Dataset1.zip
!unzip TestReal.zip

Archive:  Dataset1.zip
  inflating: Dataset/Normal/images (32)_1.jfif.jpg  
  inflating: Dataset/Normal/images (31).jfif.jpg  
  inflating: Dataset/Normal/images (31)_1.jfif.jpg  
  inflating: Dataset/Normal/images (38).jfif.jpg  
  inflating: Dataset/Normal/images (56).jfif.jpg  
  inflating: Dataset/Normal/images (28)_1.jfif.jpg  
  inflating: Dataset/Normal/images (37).jfif.jpg  
  inflating: Dataset/Normal/images (30).jfif.jpg  
  inflating: Dataset/Normal/images (29).jfif.jpg  
  inflating: Dataset/Normal/images (30)_1.jfif.jpg  
  inflating: Dataset/Normal/images (28).jfif.jpg  
  inflating: Dataset/Normal/images (41).jfif.jpg  
  inflating: Dataset/Normal/images (27).jfif.jpg  
  inflating: Dataset/Normal/images (25)_1.jfif.jpg  
  inflating: Dataset/Normal/images (26).jfif.jpg  
  inflating: Dataset/Normal/images (21).jfif.jpg  
  inflating: Dataset/Normal/images (22).jfif.jpg  
  inflating: Dataset/Normal/images (94).jpg  
  inflating: Dataset/Normal/images (27)_1.jfif.jpg  
 

In [ ]:
base_model=MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

last_layer = base_model.get_layer('block_16_project_BN')
last_output = last_layer.output

x=GlobalAveragePooling2D()(last_output)
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dropout(0.2)(x)
x=Dense(512,activation='relu')(x) #dense layer 3
x=Dropout(0.2)(x)
x=Dense(2,activation='softmax')(x) 

model = Model(base_model.input, x)

In [ ]:
base_model.summary() 
for layer in base_model.layers:
    layer.trainable=False

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [ ]:
train_datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    height_shift_range=0.2,
    shear_range=0.2,
    validation_split=0.2
) #included in our dependencies

train_generator=train_datagen.flow_from_directory('./Dataset',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)
val_generator=train_datagen.flow_from_directory('./Dataset',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True,
                                                 subset='validation')

Found 509 images belonging to 2 classes.
Found 101 images belonging to 2 classes.


In [ ]:
class MyThresholdCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshold):
        super(MyThresholdCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None): 
        accuracy = logs["val_accuracy"]
        loss = logs["val_loss"]
        if accuracy >= self.threshold and loss <= self.threshold:
            self.model.stop_training = True

In [ ]:
callback=MyThresholdCallback(threshold=0.9)

In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(train_generator,
          epochs=20,
          validation_data=val_generator)

Epoch 1/20
16/16 [==============================] - 61s 639ms/step - loss: 0.5161 - accuracy: 0.7272 - val_loss: 0.0656 - val_accuracy: 0.9703
Epoch 2/20
16/16 [==============================] - 6s 394ms/step - loss: 0.1932 - accuracy: 0.9426 - val_loss: 0.0946 - val_accuracy: 0.9604
Epoch 3/20
16/16 [==============================] - 6s 397ms/step - loss: 0.1197 - accuracy: 0.9491 - val_loss: 0.0611 - val_accuracy: 0.9901
Epoch 4/20
16/16 [==============================] - 6s 392ms/step - loss: 0.0860 - accuracy: 0.9694 - val_loss: 0.0542 - val_accuracy: 0.9802
Epoch 5/20
16/16 [==============================] - 6s 393ms/step - loss: 0.0451 - accuracy: 0.9877 - val_loss: 0.2011 - val_accuracy: 0.8812
Epoch 6/20
16/16 [==============================] - 6s 396ms/step - loss: 0.0994 - accuracy: 0.9679 - val_loss: 0.0593 - val_accuracy: 0.9703
Epoch 7/20
16/16 [==============================] - 6s 391ms/step - loss: 0.0436 - accuracy: 0.9836 - val_loss: 0.0065 - val_accuracy: 1.0000
Epoch

In [ ]:
model.evaluate(val_generator)

4/4 [==============================] - 1s 232ms/step - loss: 0.0302 - accuracy: 0.9901


[0.030201110988855362, 0.9900990128517151]

In [ ]:
def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(224, 224))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor
  
#img_path = 'C:/Users/Ferhat/Python Code/Workshop/Tensoorflow transfer learning/blue_tit.jpg'
img_path = '/content/TestReal/test/415594.jpg'
new_image = load_image(img_path)

pred = model.predict(new_image)

pred = np.argmax(pred, axis=1)

pred

In [ ]:
import pathlib

model.save("model.h5")

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model) # path to the SavedModel directory
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
tflite_model = converter.convert()

# tflite_model_file = pathlib.Path("/content/tflite")
# tflite_model_file.write_bytes(tflite_model)

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)